# Regresión de Precios de Vivienda en California con Redes Neuronales (Keras) y Regularización

**Disciplina:** Aprendizaje Profundo, Redes Neuronales, Regresión, Keras (TensorFlow)

**Objetivo:**
El objetivo de este notebook es construir, entrenar y evaluar una red neuronal para predecir los precios medianos de las viviendas en California utilizando Keras. Se incorporarán técnicas de preprocesamiento, regularización (L2 y Dropout) y callbacks de Keras (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau) para mejorar el entrenamiento y la robustez del modelo de regresión.

## 1. Carga de Librerías y Configuración Inicial

**Propósito de esta sección:**
Importar todas las bibliotecas necesarias y configurar el entorno para el análisis, incluyendo la fijación de semillas para reproducibilidad.

**Bibliotecas Clave:**
* **`numpy`, `pandas`**: Para manipulación de datos.
* **`matplotlib.pyplot`, `seaborn`**: Para visualizaciones.
* **`sklearn.datasets`**: Para cargar el dataset California Housing.
* **`sklearn.model_selection`**: Para `train_test_split`.
* **`sklearn.preprocessing`**: Para `StandardScaler`.
* **`sklearn.metrics`**: Para `mean_squared_error`, `mean_absolute_error`, `r2_score`.
* **`tensorflow.keras`**: Para construir y entrenar la red neuronal.

In [1]:
# Comandos mágicos de IPython (opcional en scripts)
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# Importación de bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Configuración para reproducibilidad
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Configuración de estilo y visualización
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

2025-05-16 18:33:23.328949: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-16 18:33:23.597948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747431203.696393   16278 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747431203.725292   16278 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747431203.953031   16278 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## 2. Funciones Personalizadas

### Descripción de la Función: `cargar_y_preparar_datos_california`

**Objetivo Principal:**
Cargar el dataset California Housing, realizar preprocesamiento (escalado de características y, opcionalmente, del objetivo) y dividirlo en conjuntos de entrenamiento y prueba.

**Características:**
* **Procesamiento:**
    1. Carga el dataset California Housing.
    2. Crea un DataFrame de Pandas para exploración.
    3. Separa características (X) y objetivo (y).
    4. Escala las características X (y opcionalmente y) usando `StandardScaler`.
    5. Divide los datos en conjuntos de entrenamiento y prueba.
* **Valor de Retorno:**
    * `X_train, X_test, y_train, y_test`: Conjuntos de datos divididos y preprocesados.
    * `scaler_X`, `scaler_y` (opcional): Los objetos `StandardScaler` ajustados.
    * `feature_names`: Nombres de características.

In [3]:
def cargar_y_preparar_datos_california(test_size=0.2, random_state=SEED, scale_target=False):
    """
    Carga, preprocesa y divide el dataset California Housing.
    """
    print("Cargando y preparando el dataset California Housing...")
    housing = fetch_california_housing()
    X = housing.data
    y = housing.target
    feature_names = housing.feature_names

    df = pd.DataFrame(X, columns=feature_names)
    df['MedHouseVal'] = y
    print("\nPrimeras filas del dataset California Housing:")
    print(df.head())
    print("\nDescripción del dataset:")
    print(df.describe().T)

    # Escalar características
    scaler_X = StandardScaler()
    X_scaled = scaler_X.fit_transform(X)
    
    y_scaled = y # Por defecto, no escalar y
    scaler_y = None
    if scale_target:
        scaler_y = StandardScaler()
        y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten() # Reshape para scaler y luego flatten

    # Dividir datos
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_scaled, test_size=test_size, random_state=random_state
    )
    
    print(f"\nDimensiones: X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")
    return X_train, X_test, y_train, y_test, scaler_X, scaler_y, feature_names

### Descripción de la Función: `crear_modelo_regresion_keras`

**Objetivo Principal:**
Definir y compilar un modelo de red neuronal secuencial con Keras para regresión.

**Características:**
* **Entrada:**
    * `input_dim` (int): Número de características de entrada.
* **Procesamiento:**
    1. Crea un modelo `Sequential`.
    2. Añade capas `Dense` con activación 'relu', regularización L2 y Dropout.
       (Considerar `BatchNormalization` opcionalmente).
    3. Añade una capa de salida `Dense` con 1 neurona (sin activación o 'linear') para regresión.
    4. Compila el modelo con optimizador 'adam', pérdida 'mean_squared_error' (MSE) y métricas 'mean_absolute_error' (MAE).
* **Valor de Retorno:**
    * `model` (tf.keras.Model): El modelo Keras compilado.

In [4]:
def crear_modelo_regresion_keras(input_dim, l2_lambda=0.001, dropout_rate=0.3):
    """
    Crea un modelo de red neuronal para regresión con Keras.
    """
    print("\nCreando el modelo de regresión con Keras...")
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,), kernel_regularizer=regularizers.l2(l2_lambda)),
        # BatchNormalization(),
        Dropout(dropout_rate),
        Dense(64, activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)),
        # BatchNormalization(),
        Dropout(dropout_rate),
        Dense(32, activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)),
        Dense(1) # Capa de salida para regresión (predice un solo valor continuo)
    ])

    model.compile(optimizer='adam',
                  loss='mean_squared_error', # MSE es común para regresión
                  metrics=['mean_absolute_error']) # MAE es otra métrica útil
    
    print("\nResumen del modelo:")
    model.summary()
    return model

### Descripción de la Función: `graficar_historial_entrenamiento_regresion`
(Similar a la de clasificación, pero para métricas de regresión)

In [5]:
def graficar_historial_entrenamiento_regresion(history):
    """
    Grafica la pérdida (MSE) y MAE durante el entrenamiento y validación.
    """
    print("\nGraficando historial de entrenamiento (Regresión)...")
    loss = history.history.get('loss')
    val_loss = history.history.get('val_loss')
    mae = history.history.get('mean_absolute_error')
    val_mae = history.history.get('val_mean_absolute_error')
    epochs_range = range(len(loss if loss else mae))

    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    if loss and val_loss:
        plt.plot(epochs_range, loss, label='Pérdida (MSE) - Entrenamiento')
        plt.plot(epochs_range, val_loss, label='Pérdida (MSE) - Validación')
        plt.title('Pérdida (MSE) de Entrenamiento y Validación')
        plt.xlabel('Épocas'); plt.ylabel('MSE')
        plt.legend(loc='upper right')

    plt.subplot(1, 2, 2)
    if mae and val_mae:
        plt.plot(epochs_range, mae, label='MAE - Entrenamiento')
        plt.plot(epochs_range, val_mae, label='MAE - Validación')
        plt.title('Error Absoluto Medio (MAE) de Entrenamiento y Validación')
        plt.xlabel('Épocas'); plt.ylabel('MAE')
        plt.legend(loc='upper right')
        
    plt.show()

### Descripción de la Función: `evaluar_y_visualizar_regresion`

**Objetivo Principal:**
Evaluar el modelo de regresión y visualizar sus predicciones.

**Características:**
* **Entrada:**
    * `model` (tf.keras.Model): Modelo Keras entrenado.
    * `X_test`, `y_test`: Datos de prueba.
    * `scaler_y` (StandardScaler, opcional): Scaler usado para el objetivo (si se escaló).
* **Procesamiento:**
    1. Evalúa el modelo (MSE, MAE).
    2. Realiza predicciones.
    3. Si `scaler_y` fue provisto, desescala `y_test` e `y_pred` para interpretación.
    4. Calcula R² score.
    5. Grafica Predicciones vs. Reales y Residuales.
* **Salida:** Muestra métricas y gráficos.

In [6]:
def evaluar_y_visualizar_regresion(model, X_test, y_test_orig, scaler_y=None):
    """
    Evalúa el modelo de regresión y visualiza predicciones y residuales.
    y_test_orig son los valores verdaderos (potencialmente escalados si scaler_y no es None).
    """
    print("\nEvaluando el modelo de regresión en el conjunto de prueba...")
    test_loss, test_mae = model.evaluate(X_test, y_test_orig, verbose=0)
    print(f"Pérdida (MSE) en el conjunto de prueba: {test_loss:.4f}")
    print(f"Error Absoluto Medio (MAE) en el conjunto de prueba: {test_mae:.4f}")

    y_pred_scaled = model.predict(X_test).flatten()
    
    y_test_final = y_test_orig
    y_pred_final = y_pred_scaled

    if scaler_y:
        print("Desescalando predicciones y valores reales para métricas e interpretación...")
        y_test_final = scaler_y.inverse_transform(y_test_orig.reshape(-1,1)).flatten()
        y_pred_final = scaler_y.inverse_transform(y_pred_scaled.reshape(-1,1)).flatten()
        
        # Recalcular MAE con valores desescalados para interpretabilidad
        mae_descalado = mean_absolute_error(y_test_final, y_pred_final)
        print(f"MAE (desescalado) en el conjunto de prueba: {mae_descalado:.4f} (en unidades originales del precio)")

    r2 = r2_score(y_test_final, y_pred_final)
    print(f"R^2 Score: {r2:.4f}")

    # Gráfico de Predicciones vs. Reales
    plt.figure(figsize=(8, 8))
    plt.scatter(y_test_final, y_pred_final, alpha=0.5)
    plt.plot([min(y_test_final), max(y_test_final)], [min(y_test_final), max(y_test_final)], '--', color='red', lw=2)
    plt.xlabel('Valores Reales')
    plt.ylabel('Predicciones')
    plt.title('Predicciones vs. Valores Reales')
    plt.grid(True)
    plt.show()

    # Gráfico de Residuales
    residuals = y_test_final - y_pred_final
    plt.figure(figsize=(8, 6))
    sns.histplot(residuals, kde=True)
    plt.xlabel('Residuales (Real - Predicción)')
    plt.ylabel('Frecuencia')
    plt.title('Distribución de los Residuales')
    plt.axvline(0, color='red', linestyle='--')
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(8, 6))
    plt.scatter(y_pred_final, residuals, alpha=0.5)
    plt.xlabel('Valores Predichos')
    plt.ylabel('Residuales')
    plt.title('Residuales vs. Valores Predichos')
    plt.axhline(0, color='red', linestyle='--')
    plt.grid(True)
    plt.show()

## 3. Desarrollo del Ejercicio: Regresión de Precios de Vivienda con Keras

### 3.1. Carga y Preparación de Datos

Cargamos el dataset California Housing, lo escalamos y dividimos.

In [7]:
# scale_target=True puede ayudar si la distribución de y es muy amplia o sesgada.
# Para este ejemplo, empecemos con scale_target=False para simplificar la interpretación inicial del MAE.
# Si el rendimiento no es bueno, se puede probar escalar el objetivo.
X_train_housing, X_test_housing, y_train_housing, y_test_housing, scaler_X_housing, scaler_y_housing, housing_feature_names = \
    cargar_y_preparar_datos_california(scale_target=False)

Cargando y preparando el dataset California Housing...

Primeras filas del dataset California Housing:
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  MedHouseVal  
0    -122.23        4.526  
1    -122.22        3.585  
2    -122.24        3.521  
3    -122.25        3.413  
4    -122.25        3.422  

Descripción del dataset:
               count         mean          std         min         25%  \
MedInc       20640.0     3.870671     1.899822    0.499900    2.563400   
HouseAge     20640.0    28.639486    12.585558    1.000000   18.000000   
AveRooms     20640.0    

### 3.2. Creación del Modelo de Regresión

Definimos la arquitectura de nuestra red neuronal para la tarea de regresión.

In [8]:
input_dim_housing = X_train_housing.shape[1]

modelo_housing = crear_modelo_regresion_keras(input_dim_housing, l2_lambda=0.01, dropout_rate=0.2)


Creando el modelo de regresión con Keras...


/home/santiago/anaconda3/envs/MADSI/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1747431211.887400   16278 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9



Resumen del modelo:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,521 (45.00 KB)

 Trainable params: 11,521 (45.00 KB)

 Non-trainable params: 0 (0.00 B)

### 3.3. Entrenamiento del Modelo

Entrenamos el modelo de regresión, utilizando callbacks.

In [9]:
print("\nIniciando el entrenamiento del modelo de regresión California Housing...")

# Callbacks
early_stopping_reg = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1) # Más paciencia para regresión
model_checkpoint_reg = ModelCheckpoint('best_housing_model.keras', monitor='val_loss', save_best_only=True, verbose=1)
reduce_lr_reg = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001, verbose=1)

callbacks_list_reg = [early_stopping_reg, model_checkpoint_reg, reduce_lr_reg]

# Entrenamiento
history_housing = modelo_housing.fit(
    X_train_housing, y_train_housing,
    epochs=250, # Un número alto de épocas
    batch_size=32,
    validation_split=0.2, # Usar una porción de los datos de entrenamiento para validación interna
    callbacks=callbacks_list_reg,
    verbose=1 
)

# modelo_housing = keras.models.load_model('best_housing_model.keras') # Si EarlyStopping no tiene restore_best_weights=True


Iniciando el entrenamiento del modelo de regresión California Housing...
Epoch 1/250


I0000 00:00:1747431213.653128   16388 service.cc:152] XLA service 0x7f308400b680 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747431213.653172   16388 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2025-05-16 18:33:33.693152: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
E0000 00:00:1747431213.865714   16388 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
E0000 00:00:1747431213.939073   16388 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but s

FailedPreconditionError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/santiago/anaconda3/envs/MADSI/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/santiago/anaconda3/envs/MADSI/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/santiago/anaconda3/envs/MADSI/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/santiago/anaconda3/envs/MADSI/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/santiago/anaconda3/envs/MADSI/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/home/santiago/anaconda3/envs/MADSI/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/home/santiago/anaconda3/envs/MADSI/lib/py

### 3.4. Visualización del Historial de Entrenamiento

Observamos cómo evolucionaron la pérdida (MSE) y el MAE.

In [ ]:
if history_housing:
    graficar_historial_entrenamiento_regresion(history_housing)

### 3.5. Evaluación del Modelo

Evaluamos el rendimiento del modelo de regresión en el conjunto de prueba.

In [ ]:
if modelo_housing:
    evaluar_y_visualizar_regresion(modelo_housing, X_test_housing, y_test_housing, scaler_y_housing)

## 4. Conclusiones del Ejercicio (Regresión California Housing)

**Resumen de Hallazgos:**
* Se cargó y preprocesó el dataset California Housing, escalando las características de entrada.
* Se construyó una red neuronal secuencial con Keras para regresión, incluyendo:
    * Capas `Dense` con activación ReLU.
    * Regularización L2 (`kernel_regularizer`) y capas `Dropout` para mitigar el sobreajuste.
    * Una capa de salida `Dense` lineal con una sola neurona para predecir el valor continuo.
* El modelo fue compilado con el optimizador `adam`, función de pérdida `mean_squared_error` (MSE) y la métrica `mean_absolute_error` (MAE).
* Se utilizaron callbacks (`EarlyStopping`, `ModelCheckpoint`, `ReduceLROnPlateau`) durante el entrenamiento para mejorar la eficiencia y el rendimiento.
* El modelo alcanzó un Error Absoluto Medio (MAE) en el conjunto de prueba de **[Completar con MAE obtenido, ej: $0.35, lo que significaría un error promedio de $35,000 si el target está en cientos de miles]** y un R² de **[Completar con R² obtenido, ej: 0.80]**.
* Las curvas de aprendizaje (MSE y MAE vs. épocas) y los gráficos de predicciones vs. reales y de residuales ayudaron a evaluar el ajuste y el comportamiento del modelo.

**Aprendizaje General:**
Este ejercicio ilustró la aplicación de redes neuronales profundas con Keras para un problema de regresión. Se demostró la importancia del preprocesamiento de datos, la inclusión de técnicas de regularización para construir modelos más generalizables, y el uso de callbacks para un entrenamiento robusto. La evaluación mediante múltiples métricas y visualizaciones es crucial para entender el rendimiento de un modelo de regresión.

*(Nota: Los resultados específicos deben completarse después de ejecutar completamente el notebook.)*